In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-classification-finetuning/sample_submission.csv
/kaggle/input/llm-classification-finetuning/train.csv
/kaggle/input/llm-classification-finetuning/test.csv


In [2]:
train_df = pd.read_csv('/kaggle/input/llm-classification-finetuning/train.csv')
test_df = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')

In [3]:
train_df['winner_model_a']

0        1
1        0
2        0
3        1
4        0
        ..
57472    1
57473    1
57474    1
57475    0
57476    1
Name: winner_model_a, Length: 57477, dtype: int64

In [4]:
#finding a method to include all 3 targets into a single thing

#making a function to get the index of every 1 value for a specific column so later on we can get the index for every model s win
def check_win(list):
    index = []
    for i in range(0, len(list)):
        if list[i] == 1:
            index.append(i)
    return index

index_wins_a = check_win(train_df['winner_model_a']) #get index for every model a win
index_wins_b = check_win(train_df['winner_model_b']) #get index for every model b win
index_ties = check_win(train_df['winner_tie']) #get index for every tie


In [5]:
#a function to make a list with all the win values coded where 1 - a, 2 - b ,0 - tie
def winner_column(index_a, index_b, index_tie):
    which_winner = ['0'] * len(train_df['id'])
    for i in index_a:
        which_winner[i] = 1 #meaning a won
    for i in index_b:
        which_winner[i] = 2 #meaning b won
    for i in index_tie:
        which_winner[i] = 0 #meaning tie
    return which_winner

train_df['which_winner'] = winner_column(index_wins_a, index_wins_b, index_ties)

In [6]:
def concat(prompt, model_a, response_a, model_b, response_b):
    return (
        f"Prompt: {prompt}\n"
        f"Response A ({model_a}): {response_a}\n"
        f"Response B ({model_b}): {response_b}"
    )

train_df["text_input"] = train_df.apply(lambda row: concat(
    row["prompt"],
    row["model_a"],
    row["response_a"],
    row["model_b"],
    row["response_b"]
), axis=1)


In [7]:
train_df = train_df.drop(columns = ['model_a', 'model_b', 'prompt', 'response_a', 'response_b', 'winner_model_a', 'winner_model_b', 'winner_tie'])

In [8]:
train_df.head()

,id,which_winner,text_input
0,30192,1,"Prompt: [""Is it morally right to try to have a..."
1,53567,2,"Prompt: [""What is the difference between marri..."
2,65089,0,"Prompt: [""explain function calling. how would ..."
3,96401,1,"Prompt: [""How can I create a test set for a ve..."
4,198779,2,"Prompt: [""What is the best way to travel from ..."


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words = 'english')

text_vect = vectorizer.fit_transform(train_df['text_input'])
print('training material has been vectorized')

training material has been vectorized


In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression

#model training has started
rf = LogisticRegression(max_iter = 300)
rf.fit(text_vect, train_df['which_winner'])
#Model has been trained

KeyboardInterrupt: 

In [ ]:
test_df.head()

In [ ]:
test_df["text_input"] = test_df.apply(lambda row: concat(
    row["prompt"],
    ' ',
    row["response_a"],
    ' ',
    row["response_b"]
), axis=1)


In [ ]:
test_df.drop(columns = ['prompt', 'response_a', 'response_b'])

In [ ]:
test_vect = vectorizer.transform(test_df['text_input'])
#test material has been vectorized

In [ ]:
test_df['which_winner'] = rf.predict(test_vect)

In [ ]:
test_df.head()

In [ ]:
def winner_to_cols(winner_column):
    winner_a = [0] * len(winner_column)
    winner_b = [0] * len(winner_column)
    winner_tie = [0] * len(winner_column)
    for i in range (0, len(winner_column)):
        if winner_column[i] == 0:
            winner_tie[i] = 1
        if winner_column[i] == 1:
            winner_a[i] = 1
        if winner_column[i] == 2:
            winner_b[i] = 1
    return winner_a, winner_b, winner_tie

a, b, tie = winner_to_cols(test_df['which_winner'])
test_df['winner_model_a'] = a
test_df['winner_model_b'] = b
test_df['winner_tie'] = tie

In [ ]:
submission = pd.DataFrame({
    'id' : test_df['id'],
    'winner_model_a' : test_df['winner_model_a'],
    'winner_model_b' : test_df['winner_model_b'],
    'winner_tie' : test_df['winner_tie']
})
submission.head()

In [ ]:
submission.to_csv('submission.csv', index = False)